In [272]:
import numpy as np
from scipy import sparse
import warnings
from numpy import linalg
warnings.filterwarnings("ignore",category= DeprecationWarning)
warnings.filterwarnings("ignore",category= FutureWarning)

In [273]:
def solve(A, q, b, 
          idx_ineqs = [],
          verbose=True, 
          precision=10, 
          max_tries=3):
    
    c = 0
    d_ = np.inf
    while True:
        p = (b / (A @ q))
        idx_ineqs_ok = np.where(p[idx_ineqs] > 1 + 1e-10)
        p[idx_ineqs_ok] = 1
        d = np.linalg.norm((b - (A @ q))[[idx for idx in range(A.shape[0]) if idx not in idx_ineqs_ok]])
        if verbose: print(f"{d:.2f}", end=' ')
        if d >= d_ - 0.01:
            c += 1
        d_ = d
        if (c == max_tries) | (d_ < precision):
            break
        A_= A.multiply(p[:, np.newaxis])
        P = np.squeeze(np.array(np.true_divide(A_.sum(0), (A_!=0).sum(0))))
        q = q * P
        if np.isnan(d):
            print('NaN error. Hay 0s en q, probable restricción inválida.')
            break
    return q

In [322]:
def check_sol(A, q, b, q_0, show=True, th1=1, th2=0.3):
    """Muestra un resumen informativo del resultado
    A: Matriz binaria de restricciones
    q: vector a optimizar, en ppio es WarmStart
    b: vector con los objetivos de las restricciones (incluye ineqs)
    """
    
    dif = (q - q_0) / q_0
    
    data = (np.linalg.norm(dif, ord=np.inf), 
            dif.max(), 
            dif.min(),
            dif.mean(),   
            np.sum(np.abs(dif) > th1),
            np.sum(np.abs(dif) > th2),
            np.sum(q < 0),
            np.linalg.norm(A @ q - b))
    
    names = f'Norm dif, Max, Min, Media, Mayores a {th1:.2f}' \
            f', Mayores a {th2:.2f}, Negativos, Norm Aq-b'.split(', ')
    
    data = {k:v for k,v in zip(names, data)}
    
    if show:
        sns.distplot(dif, kde=False, norm_hist=True)
        plt.xlim(data['Min'] - 0.1, data['Max'] + 0.1)  
        print(*[str(t[0]) + ' ' + re.search(r"[\d\-]+\.*0*[.1-9]{0,2}", 
                str(t[1])).group(0) for t in data.items()], sep = '\n')

    return data

In [274]:
A = sparse.csc_matrix([
              [1, 1, 0, 0, 0, 0, 0, 0, 0, 0], 
              [1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
              [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]]).astype('int8')

q = np.array([10, 20, 40, 50, 10, 0, 20, 20, 60, 20])
b = np.array([0.3*300, 100, 200])

In [275]:
q_1 = solve(A,q,b)
b / (A @ q_1), b / (A @ q)

115.33 28.47 17.49 10.98 6.97 

(array([1.065, 0.986, 0.980]), array([3.000, 0.714, 1.818]))

# Aleatorio

In [369]:
max_q = 100
max_b = 10000
n, m = 100, 1000

A = sparse.csc_matrix(np.random.choice([0, 1], size=(n, m)))
q = np.random.randint(1, max_q, size=m)
b = np.random.randint(1, max_b, size=n)

In [370]:
from numpy import linalg
sol_lstsq, res, rank, s = linalg.lstsq(A.toarray(), b)

In [371]:
import scipy.optimize
sol_ = scipy.optimize.minimize(lambda x: np.linalg.norm(A @ x - b, ord=np.inf), q, bounds = [(0, None) for i in range(m)])
sol_sci = sol_['x']

In [372]:
sol_ = solve(A,q,b, precision=0, max_tries=100, verbose=False)

In [375]:
for s in [sol_, sol_sci, sol_lstsq]:
    data = check_sol(A=A, q=s, b=b, q_0=q, show=False)
    print(data)
    print(s[:100])
    print('\n')

{'Norm dif': 71.23347230891879, 'Max': 71.23347230891879, 'Min': -1.0, 'Media': -0.6914103303318957, 'Mayores a 1.00': 21, 'Mayores a 0.30': 994, 'Negativos': 0, 'Norm Aq-b': 20096.939898644312}
[0.000 0.000 0.000 0.339 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 107.714
 1146.102 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 45.432 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 0.000 209.300 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 227.025 0.000 0.000
 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000
 0.000 0.000 2.256 0.000 0.000]


{'Norm dif': 1.0, 'Max': 0.0, 'Min': -1.0, 'Media': -0.8168338384052342, 'Mayores a 1.00': 0, 'Mayores a 0.30': 837, 'Negativos': 0, 'Norm Aq-b': 30046.08093784216}
[0.000 0.000 0.000

In [269]:
A = sparse.csc_matrix([
              [1, 1, 0], 
              [1, 0, 1],
              [0, 1, 1]]).astype('int8')

b = np.array([4, 2, 10])
q = linalg.inv(A.toarray()) @ np.array([2,1,5])

In [270]:
q_1 = solve(A,q,b)
b / (A @ q_1), b / (A @ q)

5.48 

(array([2.000, 2.000, 2.000]), array([2.000, 2.000, 2.000]))

In [3]:
R = np.array([[1,0,1],[1,0,0],[0,1,1]])
e = np.array([0.9, 0.5, 1.5])

In [4]:
#np.mean(R, axis=1)
(R @ [1,1,1])

array([2, 1, 2])

In [5]:
import scipy.linalg as linalg

In [6]:
sol, res, rank, s = linalg.lstsq(R, b)

In [7]:
o = q - sol

In [8]:
A = linalg.null_space(R)

In [9]:
p =  A @ (A.T @ o) 

In [10]:
sol + p

array([100., 210., -10.])

In [3]:
#A @ q

In [4]:
# Ejemplo: tenemos 5 clientes con 2 productos cada uno
# Las restricciones serán: el cliente 1 no puede tener mas del 30% del total 
# el presupuestp del producto 1 es 100
# el presupuesto del producto 2 es 200

# Tenemos entonces un vector binario de largo q = clientes * productos
# Cada fila representa: (cl1) p1 p2 (cl2) p1 p2 (cl3) p11 pl2 (cl4) p1 p2 (cl5) p1 p2

In [5]:
q_ = q.copy()

In [6]:
# resultado anterior, diferencia
A @ q_, A @ q_ - b, 

(array([110., 110., 170.]), array([ 20.,  10., -30.]))

In [8]:
# Tenemos distintas posibilidades:
# 1) recorrer las restricciones en orden y llevar cada una a 0. Este proceso tiene dos partes: 1) forward, llevar el grupo a 0. 2) back, llevar el resto a la misma cantidad
# 2) tomar la restriccion más insatisfecha, llevar esa a 0 y repetir
# 3) usar el vector de diferencias para multiplicar cada subconjunto por un lr diferencial ponderado por la diferencia a esa restriccion

In [11]:
from sqlalchemy import inspect

In [ ]:
for table_name in inspector.get_table_names():
    print(table_name)

In [71]:
TMP_PATH = r'C:\Users\mgrinberg\Desktop\disc c\SDPC\SDPC\tmp\\'
from scipy import sparse
import numpy as np
import pandas as pd

In [93]:
A, x, b, idx_ineqs = [TMP_PATH + p for p in ['A.npz', 'x.gz', 'b.gz', 'idx_ineqs.gz']]
A = sparse.load_npz(A).astype(bool)
x = np.loadtxt(x).astype('float64')
b = np.loadtxt(b).astype('int32')
idx_ineqs = np.loadtxt(idx_ineqs).astype('int32')
print('Datos cargados.')

Datos cargados.


In [89]:
dif = ([1/10] + list(np.ones(b.shape[0] - 1)))

b_ = b * dif

In [94]:
write_sol(x)
validar()

Guardando resultado en Q1.csv
Volcando resultado en Análisis.
Ingresando resultados en Restricciones.
Errores en restricciones con indices [  3  14  22  36  54  55  64  72  80  93 104 105 117 131 138 140 158 166
 167 172 176]


In [87]:
q = x
lr = 0.01
epochs = 10
lim=10
verbose=False
cvx = False
polish = False
precision=0.01
live = False

In [83]:
b = b_
i = 0
rows = A.shape[0]
idx_eqs = [i for i in range(rows) if i not in idx_ineqs]
while True:
    if np.all(abs(b[idx_eqs] - A[idx_eqs] @ x) < abs(b[idx_eqs])*precision) & np.all(A[idx_ineqs] @ x <= (b[idx_ineqs] * (1 + precision))):
        print('Todas las restricciones satisfechas.')
        break   
    print('Epoch: ', j)
    for i in range(rows):
        a = A[i].toarray().ravel()
        q[a] *= (b[i]/(q @ a))
        print(i, b[i] - (q @ a))

Todas las restricciones satisfechas.


In [3]:
def solve_cvx(A, x, b, idx_ineqs, solver = 'SUPERSCS', max_iters=100, eps = 1e-09, scale=1):
    n = x.shape[0]
    q1 = cp.Variable(n, nonneg=True)
    #warm start 
    q1.value = x

    constrains=[]
    for i in range(A.shape[0]):
        if i in idx_ineqs:
            constrain = cp.sum(q1[A[i].toarray().ravel()]) <= int(b[i])
        else:
            constrain = cp.sum(q1[A[i].toarray().ravel()]) == int(b[i])        
        constrains.append(constrain)
    
    print('Constrains loaded.')
    
    obj = cp.Minimize(cp.norm(q1-x, p=1))
    #obj = cp.Minimize(cp.norm(cp.kl_div(x, q1)))# - cp.sum(q1)))
    prob = cp.Problem(obj, constrains)
    print("Problem made.")

    # liberar memoria
    del A, x, b

    if solver == 'OSQP':
        sol = prob.solve(solver, warm_start = True, verbose = True, max_iter=max_iters, linsys_solver='mkl pardiso')
    elif 'SCS' in solver:
        sol = prob.solve(solver, warm_start = True, verbose = True, max_iters=max_iters, eps=eps, scale=scale)
    print("Problem solved.")

    return q1.value, sol